# High-Solar-Exposure (HSE) analysis

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
from alive_progress import alive_it
from pathlib import Path

# Ignore FutureWarnings from geopandas
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', message='.*initial implementation of Parquet.*')
# Ignore SettingWithCopyWarning from (geo-)pandas
pd.options.mode.chained_assignment = None  # default='warn'


# Config
day = 170
sensitivity_factor = 1.0

in_dir = Path(f'../../export/{day}/{sensitivity_factor}/exportdata')
out_dir = Path(in_dir.parent / 'analysis/segment_level')
out_dir.mkdir(parents=True, exist_ok=True)

tod_list = ['10am', '1pm', '4pm', '7pm']
colors = {
    '10am': 'yellowgreen',
    '1pm': 'darkgoldenrod',
    '4pm': 'plum',
    '7pm': 'tab:blue',
}
default_type = 'shortest'
optimized_type = 'shaded'
custom_labels = [default_type.title(), 'Shaded']

high_sol_expo = 90
low_sol_expo = 50

n_routes = 625 * 16 # config file parameters (n_routes * poi types)

sns.set_theme(style='whitegrid')
sns.set_context('paper')

xlim = (-2, 102)
ylim = (-2, 102)
ylim_neg=(2, -102)

In [2]:
# Helper functions
def tod_format(tod):
    """Returns the formatted time of day string"""
    return tod[:-2] + ' ' + tod[-2:].upper()

## Calculate HSE length and share per tod

In [ ]:
hse_data = []

for tod in tod_list:
    print(f'Processing {tod} data...')
    gdf = gpd.read_feather(in_dir / f'segments/segments_{tod}.feather')
    gdf.to_crs(epsg=25832, inplace=True)
    
    for route_i in alive_it(range(1, n_routes + 1)):
        gdf_ex = gdf[gdf['trip_id'] == route_i]
        total_length = gdf_ex['geom'].length.sum()

        gdf_hse = gdf_ex[gdf_ex[f'sol_expo_{tod}'] >= high_sol_expo]
        hse_length = gdf_hse['geom'].length.sum()

        hse_share = (hse_length / total_length) * 100

        route_data = {
            'trip_id': route_i,
            'time_of_day': tod,
            'route_type': f'{optimized_type} route',
            'length': round(total_length, 2),
            'hse_length': round(hse_length, 2),
            'hse_share': round(hse_share, 2)
        }
        hse_data.append(route_data)

### Add default data

In [48]:
gdf = gpd.read_feather(in_dir / f'segments/segments_{default_type}.feather')
gdf.to_crs(epsg=25832, inplace=True)

In [49]:
hse_data_default = []

In [ ]:
for route_i in alive_it(range(1, n_routes + 1)):
    gdf_ex = gdf[gdf['trip_id'] == route_i]
    total_length = gdf_ex['geom'].length.sum()

    for tod in tod_list:
        gdf_hse = gdf_ex[gdf_ex[f'sol_expo_{tod}'] >= high_sol_expo]
        hse_length = gdf_hse['geom'].length.sum()

        hse_share = (hse_length / total_length) * 100

        route_data = {
            'trip_id': route_i,
            'time_of_day': tod,
            'route_type': f'{default_type} route',
            'length': round(total_length, 2),
            'hse_length': round(hse_length, 2),
            'hse_share': round(hse_share, 2)
        }
        hse_data_default.append(route_data)

### Postprocess HSE data

In [ ]:
len(hse_data), len(hse_data_default)

In [ ]:
hse_df = pd.DataFrame(hse_data + hse_data_default)
hse_df.sort_values(by=['trip_id', 'time_of_day'], inplace=True)
hse_df.reset_index(drop=True, inplace=True)
hse_df

In [53]:
hse_df.to_feather(out_dir / 'hse_segments.feather')

## Plotting

In [5]:
hse_df = pd.read_feather(out_dir / 'hse_segments.feather')

### HSE Share

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows=1, figsize=(12, 5), sharey=True)

y_value = 'hse_share'
y_label = 'Share of HSE segments [%]'

for col, (tod, ax) in enumerate(zip(tod_list, axes)):
    df_ex = hse_df[hse_df['time_of_day'] == tod]
    g = sns.boxenplot(data=df_ex, x='route_type', y=y_value, order=[f'{default_type} route', f'{optimized_type} route'], palette={f'{default_type} route': 'darkgrey', f'{optimized_type} route': colors[tod]},  ax=ax)
    g.set_xlabel('')
    g.set_ylabel(y_label, fontsize=12) if col == 0 else g.set_ylabel('')
    g.set_ylim(ylim)
    g.set_title(tod_format(tod), fontsize=12)
    median_default = df_ex[df_ex['route_type'] == f'{default_type} route'][y_value].median()
    median_time = df_ex[df_ex['route_type'] == f'{optimized_type} route'][y_value].median()
    max_default = df_ex[df_ex['route_type'] == f'{default_type} route'][y_value].max()
    max_time = df_ex[df_ex['route_type'] == f'{optimized_type} route'][y_value].max()
    min_default = df_ex[df_ex['route_type'] == f'{default_type} route'][y_value].min()
    min_time = df_ex[df_ex['route_type'] == f'{optimized_type} route'][y_value].min()
    g.text(0, median_default, f'{round(median_default, 2)}', ha='center', va='center', fontweight='bold', fontsize=10, color='white')
    g.text(1, median_time, f'{round(median_time, 2)}', ha='center', va='center', fontweight='bold', fontsize=10, color='white')
    g.set_xticklabels(custom_labels, fontsize=12)

    print(f'{tod} - {y_value} - {default_type} - Median: {median_default} - Max: {max_default} - Min: {min_default}')
    print(f'{tod} - {y_value} - {optimized_type} - Median: {median_time} - Max: {max_time} - Min: {min_time}')

plt.tight_layout()
plt.savefig(out_dir / f'{y_value}.png', dpi=300, bbox_inches='tight')
plt.show()